In [13]:
import re
import glob
import os
import pandas as pd
import pdfplumber
from collections import defaultdict


def extract_filing_id(text):
    """从文本中提取Filing ID"""
    match = re.search(r'FilingId=(\d+)', text)
    return match.group(1) if match else ''


def extract_metadata(pdf_path):
    """从第一页提取元数据"""
    with pdfplumber.open(pdf_path) as pdf:
        text = pdf.pages[0].extract_text()
        
        # 检查是否有编码问题
        if '(cid:' in text or not text or len(text) < 50:
            raise ValueError("PDF存在编码问题，无法读取")
        
        # 合并为单行，处理跨行标题
        text_single_line = ' '.join(text.split('\n'))
        
        # 查找标题行，兼容NMLS ID换行情况，不限制公司名格式
        title_match = re.search(r'(\d{4}\s+Q\d.*?for.*?\([\d\s]+\))', text_single_line)
        
        if not title_match:
            raise ValueError("无法匹配标题行")
        
        title_line = title_match.group(1)

        # 兼容公司名称含'(数字)'情况
        all_nmls = re.findall(r'\(([\d\s]+)\)', title_line)
        Company_NMLS_ID = all_nmls[-1] if all_nmls else None
    
    # 提取各字段
    metadata = {
        'Company NMLS ID': Company_NMLS_ID,
        'Company Name': re.search(r'for\s+(.*?)\s+\(', title_line).group(1).strip(),
        'Fiscal Year': re.search(r'(\d{4})\s+Q\d', title_line).group(1),
        'Qtr': re.search(r'\d+\s+(Q\d)', title_line).group(1),
        'Form Type': re.search(r'Q\d\s+(.*?)\s+V\d', title_line).group(1).strip(),
        'Filing ID': extract_filing_id(text)
    }
    
    return metadata


def extract_credit_lines(pdf_path):
    """从General提取CreditLine"""
    with pdfplumber.open(pdf_path) as pdf:
        text = '\n'.join([page.extract_text() for page in pdf.pages])
    
    credit_lines = []
    # CreditLine贪婪匹配，兼容负值、小数和Name of Provider含数字情况
    pattern = r'^(\d+)\s+(.+)\s+(-?[\d,]+(?:\.\d+)?)\s+(-?[\d,]+(?:\.\d+)?)\s*$'
    
    for line in text.split('\n'):
        match = re.match(pattern, line.strip())
        if match:
            credit_lines.append({
                'Record ID': int(match.group(1)),
                'Name of Provider': match.group(2).strip(),
                'Credit Limit($)': float(match.group(3).replace(',', '')),
                'Remaining Credit Period End($)': float(match.group(4).replace(',', ''))
            })
    
    return credit_lines


def extract_service_data(pdf_path):
    """从General提取Service"""
    LS_CODES = ['LS010', 'LS020', 'LS030', 'LS040', 'LS090',
                'LS100', 'LS110', 'LS190',
                'LS200', 'LS210', 'LS220', 'LS230', 'LS290']
    
    with pdfplumber.open(pdf_path) as pdf:
        full_text = '\n'.join([page.extract_text() for page in pdf.pages])
    
    service_data = {}
    
    # 提取每个LS代码的值
    for code in LS_CODES:
        # ServiceLine匹配，兼容负值情况（不兼容空值）
        pattern = rf'{code}\s+.*?\s+(-?[\d,]+(?:\.\d+)?)\s+(-?[\d,]+)\s+(-?[\d,]+(?:\.\d+)?)'
        match = re.search(pattern, full_text)
        
        if match:
            service_data[f'{code} UPB($)'] = float(match.group(1).replace(',', ''))
            service_data[f'{code} Loan Count(#)'] = int(match.group(2).replace(',', ''))
            service_data[f'{code} Avg Loan Sz($)'] = float(match.group(3).replace(',', ''))
    
    # 提取NOTE
    lines = [line.strip() for line in full_text.split('\n')]
    note_content = []
    collecting = False
    trigger_key = "NOTE Explanatory Notes - Free Text"

    # 按行查找NOTE
    for line in lines:
        if trigger_key in line:
            # 截取之后的文本
            suffix_text = line.split(trigger_key)[-1].strip()
            # 文本非空则添加到NOTE列表
            if suffix_text:
                note_content.append(suffix_text)
            collecting = True
            continue
        if 'Warnings' in line:
            collecting = False
            break
        if collecting:
            if not line:
                continue
            if line.startswith('https'): #排除页脚，查找以http开头的行
                continue
            if re.match(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', line): # 排除页眉，查找以日期开头的行
                continue
            note_content.append(line)
    
    service_data['NOTE'] = ' '.join(note_content).strip() if note_content else ''
    
    return service_data


def extract_fc_data(pdf_path, field_list):
    """
    从FC提取财务数据
    
    思路：
    1. 从field_list中提取字段代码，统计每个代码对应几个字段
    2. 查找PDF中该字段代码的行
    3. 从后向前匹配N个数值（N=字段数量）
    4. 按顺序分配给对应的字段
    """
    
    # 去掉NOTE字段和文本字段，按字段代码分组
    code_to_fields = defaultdict(list)
    text_fields = ['A230J']  # 文本字段列表
    
    for field in field_list:
        if field.endswith('NOTE'):
            continue
        
        # 在第一个空格处截取，提取字段代码
        parts = field.split(' ', 1)
        if parts:
            code = parts[0]
            # 跳过文本字段
            if code in text_fields:
                continue
            code_to_fields[code].append(field)
    
    fc_data = {}
    
    # 提取文本
    with pdfplumber.open(pdf_path) as pdf:
        full_text = '\n'.join([page.extract_text() for page in pdf.pages])
    
    # 对每个字段代码，从后向前匹配数值
    for code, fields in code_to_fields.items():
        expected_count = len(fields)  # 需要匹配的数值数量
        
        # 查找该字段代码的行
        pattern = rf'^{code}\s+.*?$'
        match = re.search(pattern, full_text, re.MULTILINE) # 多行模式匹配
        
        if not match:
            continue
        
        line = match.group(0)
        
        # 提取所有数值
        all_numbers = re.findall(r'-?(?:\d+,)*\d+(?:\.\d+)?', line)
        
        # 从后向前取expected_count个数值
        values = all_numbers[-expected_count:] if len(all_numbers) >= expected_count else all_numbers
        
        # 按顺序分配给对应的字段
        for i, field in enumerate(fields):
            if i < len(values):
                value_str = values[i].replace(',', '')
                fc_data[field] = float(value_str)
    
    # 提取NOTE
    lines = [line.strip() for line in full_text.split('\n')]
    
    # 提取FCNOTE
    note_content = []
    collecting = False
    trigger_key = "FCNOTE Explanatory Notes - Free Text" # 定义触发字段
    
    for line in lines:
        if trigger_key in line:
            suffix_text = line.split(trigger_key)[-1].strip() # 截取触发字段后文本
            if suffix_text:
                note_content.append(suffix_text)
            collecting = True
            continue

        if collecting:           
            if 'Warnings' in line: # 匹配到终止词停止
                collecting = False
                break
            if not line:
                continue
            if line.startswith('https'): # 排除页脚
                continue
            if re.match(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', line): # 排除页眉
                continue
            note_content.append(line)
    
    fc_data['FCNOTE'] = ' '.join(note_content).strip() if note_content else ''

    # 提取A230J
    note_content = []
    collecting = False
    trigger_key = "A230J Explanation of Amounts in 'Other Assets Other'" # 定义触发字段
    
    for line in lines:
        if trigger_key in line:
            suffix_text = line.split(trigger_key)[-1].strip() # 截取触发字段后文本
            if suffix_text:
                note_content.append(suffix_text)
            collecting = True
            continue
 
        if collecting:
            if 'Schedule B' in line: # 匹配到终止词停止
                collecting = False
                break
            if not line:
                continue
            if line.startswith('https'): # 排除页脚
                continue
            if re.match(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', line): # 排除页眉
                continue
            note_content.append(line)
    
    fc_data['A230J Explanation'] = ' '.join(note_content).strip() if note_content else ''
    
    # 提取其他NOTE字段
    note_field_mapping = {
        'A60AHNOTE': 'A060AH',
        'A60AINOTE': 'A060AI',
        'A230GNOTE': 'A230G',
        'A230HNOTE': 'A230H',
        'B350NNOTE': 'B350N'
    }

    # 为每个NOTE字段定义其后续字段的标识
    note_stop_patterns = {
        'A060AH': r'A060AI\s+MEMO',
        'A060AI': r'Schedule\s+A-090',
        'A230G': r'A230H\s+Other\s+Assets\s+Other',
        'A230H': r'A230T\s+Total\s+Other\s+Assets',
        'B350N': r'B350T\s+Balance\s+at\s+End\s+of\s+Period'
    }
    
    for csv_field, pdf_code in note_field_mapping.items():
        trigger_key = f"{pdf_code} Explanatory Note - Free Text"
        
        note_content = []
        collecting = False
        
        for line in lines:
            if trigger_key in line:
                suffix_text = line.split(trigger_key)[-1].strip()
                if suffix_text:
                    note_content.append(suffix_text)
                collecting = True
                continue
            
            if collecting:
                # 停止条件1: Schedule
                if 'Schedule' in line:
                    break

                # 停止条件2: 特定的下一个字段
                if pdf_code in note_stop_patterns:
                    stop_pattern = note_stop_patterns[pdf_code]
                    if re.search(stop_pattern, line):
                        break

                else:
                    # 停止条件3: 匹配到其他字段代码（兜底，若NOTE中存在其它字段代码可能导致异常）
                    field_match = re.match(r'^([A-Z]{1,2}\d{3}[A-Z]{0,2})\s', line)
                    if field_match:
                        matched_code = field_match.group(1)
                        # 排除匹配到本身，提高稳健性
                        if matched_code != pdf_code:
                            break
                
                if not line:
                    continue
                if line.startswith('https'):
                    continue
                if re.match(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', line):
                    continue
                
                note_content.append(line)
        
        fc_data[csv_field] = ' '.join(note_content).strip() if note_content else ''
        
    return fc_data



def process_pdfs(pdf_folder='CR', sample_fc_path='CR_Test/FC.csv', output_folder='.'):
    """
    处理PDF并生成CSV
    
    pdf_folder：提取文件夹地址
    sample_fc_path：示例FC.csv
    """
    
    print("=" * 60)
    print("PDF2CSV")
    print("=" * 60)
    
    # 1. 根据文件名查找PDF文件
    print("\n[1/5] 扫描PDF文件...")
    general_pattern = os.path.join(pdf_folder, '*General.pdf')
    fc_pattern = os.path.join(pdf_folder, '*FC.pdf')
    
    general_files = glob.glob(general_pattern)
    fc_files = glob.glob(fc_pattern)
    
    print(f"  找到 {len(general_files)} 个General文件")
    print(f"  找到 {len(fc_files)} 个FC文件")
    
    # 2. 加载FC字段列表
    print("\n[2/5] 加载FC字段配置...")
    sample_fc = pd.read_csv(sample_fc_path, nrows=0)
    fc_field_list = [col for col in sample_fc.columns 
                     if col not in ['Company NMLS ID', 'Company Name', 'Fiscal Year', 
                                    'Qtr', 'Form Type', 'Filing ID']]
    print(f"  FC字段数量: {len(fc_field_list)}")
    
    # 3. 处理General文件
    print("\n[3/5] 处理General文件...")
    credit_line_records = []
    service_records = []
    skipped_files = []
    
    for i, general_file in enumerate(general_files, 1):
        print(f"  [{i}/{len(general_files)}] {os.path.basename(general_file)}", end='') # 打印当前处理进度+文件名
        
        try:
            # 提取数据
            metadata = extract_metadata(general_file)
            credit_lines = extract_credit_lines(general_file)
            service_data = extract_service_data(general_file)
            
            for cl in credit_lines: # 遍历当前文件全部CreditLine，合并元数据
                credit_line_records.append({**metadata, **cl})
            
            service_records.append({**metadata, **service_data})
            
            print(" ✓")
            
        except Exception as e:
            print(f" ✗ 跳过")
            print(f"      原因: {str(e)}")
            skipped_files.append((general_file, str(e)))
    
    # 4. 处理FC文件
    print(f"\n[4/5] 处理FC文件...")
    fc_records = []
    
    for i, fc_file in enumerate(fc_files, 1):
        print(f"  [{i}/{len(fc_files)}] {os.path.basename(fc_file)}", end='')
        
        try:
            # 提取数据
            metadata = extract_metadata(fc_file)
            fc_data = extract_fc_data(fc_file, fc_field_list)
            fc_records.append({**metadata, **fc_data})
            
            print(" ✓")
            
        except Exception as e:
            print(f" ✗ 跳过")
            print(f"      原因: {str(e)}")
            skipped_files.append((fc_file, str(e)))
    
    # 5. 生成CSV文件
    print(f"\n[5/5] 生成CSV文件...")
    
    if credit_line_records:
        df_credit = pd.DataFrame(credit_line_records)
        df_credit = df_credit.drop_duplicates(
            subset=['Company NMLS ID', 'Filing ID', 'Record ID', 'Name of Provider', 'Credit Limit($)', 'Remaining Credit Period End($)'], 
            keep='first'
        )
        df_credit.to_csv(os.path.join(output_folder, 'CreditLine_output.csv'), index=False)
    print(f"  ✓ CreditLine_output.csv ({len(df_credit)} 行)")
    
    if service_records:
        df_service = pd.DataFrame(service_records)
        df_service = df_service.drop_duplicates(
            subset=['Company NMLS ID', 'Filing ID'], 
            keep='first'
        )
        df_service.to_csv(os.path.join(output_folder, 'Service_output.csv'), index=False)
    print(f"  ✓ Service_output.csv ({len(df_service)} 行)")
    
    if fc_records:
        df_fc = pd.DataFrame(fc_records)
        df_fc = df_fc.drop_duplicates(
            subset=['Company NMLS ID', 'Filing ID'], 
            keep='first'
        )
        df_fc.to_csv(os.path.join(output_folder, 'FC_output.csv'), index=False)
    print(f"  ✓ FC_output.csv ({len(df_fc)} 行)")
    
    # 统计信息
    print("\n" + "=" * 60)
    print("处理完成！")
    print("=" * 60)
    
    total_files = len(general_files) + len(fc_files)
    success_files = total_files - len(skipped_files)
    
    print(f"成功处理: {success_files} / {total_files} 个文件")
    print(f"  - General: {len(service_records)} / {len(general_files)}")
    print(f"  - FC: {len(fc_records)} / {len(fc_files)}")
    
    if skipped_files:
        print(f"\n⚠️  跳过 {len(skipped_files)} 个文件:")
        for file, error in skipped_files:
            print(f"  - {os.path.basename(file)}")
            print(f"    原因: {error}")
        
        # 保存跳过的文件列表
        skipped_df = pd.DataFrame(skipped_files, columns=['File Path', 'Error'])
        skipped_output = os.path.join(output_folder, 'skipped_files.csv')
        skipped_df.to_csv(skipped_output, index=False)
        print(f"\n跳过文件列表已保存: {skipped_output}")
    else:
        print("\n✓ 所有文件处理成功！")
    
    return df_credit, df_service, df_fc, skipped_files

df_credit, df_service, df_fc, skipped_files = process_pdfs()

PDF2CSV

[1/5] 扫描PDF文件...
  找到 370 个General文件
  找到 111 个FC文件

[2/5] 加载FC字段配置...
  FC字段数量: 730

[3/5] 处理General文件...
  [1/370] Aurora Financial, LLC Q1 General.pdf ✓
  [2/370] American Heritage Capital, LP Q1 General.pdf ✓
  [3/370] American First Lending Corporation_General.pdf ✗ 跳过
      原因: PDF存在编码问题，无法读取
  [4/370] Adair Financial Services, LLC Q1 General.pdf ✓
  [5/370] American Advisors Group_General.pdf ✗ 跳过
      原因: PDF存在编码问题，无法读取
  [6/370] Academy Mortgage Corporation Q1 General.pdf ✓
  [7/370] Complete Payment Recovery Services, Inc. Q1 General.pdf ✓
  [8/370] City First Mortgage Services, L.L.C. Q1 General.pdf ✓
  [9/370] CTC Mortgage Services, LLC Q1 General.pdf ✓
  [10/370] Choice Lending Corp_General.pdf ✗ 跳过
      原因: PDF存在编码问题，无法读取
  [11/370] Change Lending, LLC_General.pdf ✗ 跳过
      原因: PDF存在编码问题，无法读取
  [12/370] Agiant Realty Services Corporation Q1 General.pdf ✓
  [13/370] Chelgren and Associates, Inc._General.pdf ✗ 跳过
      原因: PDF存在编码问题，无法读取
  [14/370] Bainbridge Le